In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dansbecker/food-101")

print("Path to dataset files:", path)

100%|██████████| 9.38G/9.38G [07:05<00:00, 23.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1


In [5]:
import os

for root, dirs, files in os.walk(path):
    if 'images' in dirs and '__MACOSX' not in root:
        dataset_path = os.path.join(root, 'images')
        print("✅ Actual dataset images folder:", dataset_path)
        break


✅ Actual dataset images folder: /root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1/food-101/food-101/images


In [7]:
dataset_path = "/root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1/food-101/food-101/images"


In [8]:
import torch
import torch.nn as nn
import torchvision.models as models

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pre-trained ResNet18 and modify final layer
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 10)  # 10 classes
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 60.2MB/s]


In [10]:
import os
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from PIL import Image
from sklearn.preprocessing import LabelEncoder



In [12]:
dataset_path = "/root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1/food-101/food-101/images"

# ✅ Filter to only directories
all_classes = sorted([
    d for d in os.listdir(dataset_path)
    if os.path.isdir(os.path.join(dataset_path, d))
])

selected_classes = all_classes[:10]  # Pick first 10
image_paths = []

for cls in selected_classes:
    class_dir = os.path.join(dataset_path, cls)
    images = [os.path.join(cls, img) for img in os.listdir(class_dir)]
    image_paths.extend(images)

random.shuffle(image_paths)

split_idx = int(0.8 * len(image_paths))
train_list = image_paths[:split_idx]
test_list = image_paths[split_idx:]

print(f"✅ Reduced dataset to {len(selected_classes)} classes.")
print("✅ Train/Test split done.")


✅ Reduced dataset to 10 classes.
✅ Train/Test split done.


In [13]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms

# Label encode the class names
all_labels = [img_path.split('/')[0] for img_path in train_list + test_list]
le = LabelEncoder()
le.fit(all_labels)

# Define image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Custom Dataset
class Food101Dataset(Dataset):
    def __init__(self, image_list, root_dir, label_encoder, transform=None):
        self.image_list = image_list
        self.root_dir = root_dir
        self.le = label_encoder
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_path = self.image_list[idx]
        full_path = os.path.join(self.root_dir, img_path)
        image = Image.open(full_path).convert("RGB")

        label_str = img_path.split('/')[0]
        label = self.le.transform([label_str])[0]

        if self.transform:
            image = self.transform(image)

        return image, label

# Initialize datasets
train_dataset = Food101Dataset(train_list, dataset_path, le, transform)
test_dataset = Food101Dataset(test_list, dataset_path, le, transform)

# Create data loaders
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("✅ Data loaders ready!")


✅ Data loaders ready!


In [14]:
import torch.nn as nn
import torchvision.models as models

# Load ResNet18 with pretrained weights
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Replace the final fully connected layer to match number of classes
num_classes = len(le.classes_)  # Should be 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"✅ Model ready on device: {device}")


✅ Model ready on device: cuda


In [15]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [38]:
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"✅ Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


✅ Epoch [1/3], Loss: 0.1143, Accuracy: 96.29%
✅ Epoch [2/3], Loss: 0.1005, Accuracy: 96.69%
✅ Epoch [3/3], Loss: 0.0357, Accuracy: 98.78%


In [39]:
model.eval()  # Set to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"✅ Test Accuracy: {accuracy:.2f}%")


✅ Test Accuracy: 74.55%


In [41]:

selected_classes = le.classes_.tolist()

print("Selected 10 food classes from LabelEncoder:")
for i, cls in enumerate(selected_classes, 1):
    print(f"{i}. {cls}")


Selected 10 food classes from LabelEncoder:
1. apple_pie
2. baby_back_ribs
3. baklava
4. beef_carpaccio
5. beef_tartare
6. beet_salad
7. beignets
8. bibimbap
9. bread_pudding
10. breakfast_burrito


In [42]:
calorie_dict = {
    "apple_pie": 296,
    "baby_back_ribs": 320,
    "baklava": 290,
    "beef_carpaccio": 180,
    "beef_tartare": 250,
    "beet_salad": 150,
    "beignets": 220,
    "bibimbap": 500,
    "bread_pudding": 300,
    "breakfast_burrito": 450
}


In [43]:
from torchvision import transforms

# Same transform used during training
predict_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [44]:
from PIL import Image
import torch

def predict_image_with_calories(image_path, model, label_encoder, calorie_info):
    model.eval()

    # Load image and preprocess
    image = Image.open(image_path).convert("RGB")
    image_tensor = predict_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)

    predicted_label = label_encoder.inverse_transform([predicted.cpu().item()])[0]

    # Get calories from dict
    calories = calorie_info.get(predicted_label, "N/A")

    return predicted_label, calories


In [45]:
import os

# Find a real image path from one of the selected classes
for cls in selected_classes:
    class_dir = os.path.join(dataset_path, cls)
    image_files = [f for f in os.listdir(class_dir) if f.endswith('.jpg')]
    if image_files:
        sample_image_path = os.path.join(class_dir, image_files[0])
        print("✅ Sample image found:", sample_image_path)
        break


✅ Sample image found: /root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1/food-101/food-101/images/apple_pie/2536578.jpg


In [46]:
# Use the sample path we found
sample_image_path = "/root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1/food-101/food-101/images/apple_pie/2536578.jpg"

# Predict
predicted_class, calories = predict_image_with_calories(sample_image_path, model, le, calorie_dict)

# Output results
print(f"🍽️ Predicted class: {predicted_class}")
print(f"🔥 Calories per serving: {calories} kcal")


🍽️ Predicted class: apple_pie
🔥 Calories per serving: 296 kcal
